# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries
import nltk

import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [7]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('Message', engine)
x = df['message']
y = df.iloc[:,4:]
col_names = y.columns.tolist()

### 2. Write a tokenization function to process your text data

In [8]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    tokens = word_tokenize(text)
    lem = WordNetLemmatizer()

    ts = []
    for token in tokens:
        t = lem.lemmatize(token).lower().strip()
        ts.append(t)

    return ts


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
def ML_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

    parameters = {
        'clf__estimator__n_estimators': [10],
        'clf__estimator__min_samples_split': [2],
    
    }
    model = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=2, cv=3)
    return model

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y)

model = ML_pipeline()
model.fit(x_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


/home/lot3sgh/projects/udacity_data_science_disaster_pipeline/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f0795b77d30>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=4,
             param_grid={'clf__estimator__min_samples_split': [2],
                         'clf__estimator__n_estimators': [10]},
             verbose=2)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = model.predict(x_test)

In [12]:
class_report = classification_report(y_test, y_pred, target_names=col_names)
class_report

/home/lot3sgh/projects/udacity_data_science_disaster_pipeline/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lot3sgh/projects/udacity_data_science_disaster_pipeline/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lot3sgh/projects/udacity_data_science_disaster_pipeline/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division

'                        precision    recall  f1-score   support\n\n               related       0.83      0.94      0.88      5038\n               request       0.83      0.39      0.53      1116\n                 offer       0.00      0.00      0.00        30\n           aid_related       0.74      0.50      0.60      2713\n          medical_help       0.68      0.08      0.15       495\n      medical_products       0.54      0.05      0.09       307\n     search_and_rescue       0.44      0.04      0.07       179\n              security       0.00      0.00      0.00       119\n              military       0.76      0.08      0.15       225\n           child_alone       0.00      0.00      0.00         0\n                 water       0.84      0.28      0.42       395\n                  food       0.82      0.39      0.53       713\n               shelter       0.81      0.33      0.47       562\n              clothing       0.65      0.12      0.21        90\n                 money

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.